In [1]:
from sas7bdat import SAS7BDAT
import pandas as pd
from datetime import date
import os
import re 
import subprocess

ModuleNotFoundError: No module named 'sas7bdat'

** JIRA BDD 407 **  
Convert sas files to txt files, tab-separated.  
Change all dates to number of days to date of vaccination as defined in ha_study_vaccination_format.sas7bdat.


In [ ]:
beg = 'Please enter file path for '
sasfile_day0 = input(beg + 'sasfile: \n')
day0file = input(beg + "day 0 file: \n")



In [ ]:
### Start with the file that has the vaccine dates to use as day 0
#sasfile_day0 = "/home/cthomas/Desktop/cris/projects/BDD407/converted_sasfiles/ha_study_vaccination_format.sas7bdat"
#day0file = "/home/cthomas/Desktop/cris/projects/BDD407/ha_study_vaccination_format.txt"

with SAS7BDAT(sasfile_day0) as f:
    df = f.to_data_frame()
df.to_csv(day0file,sep="\t")
df.head()

In [ ]:
## Get day 0 for all subjects:
day0 = {}
with open(day0file, "r") as day0f:
    fl = day0f.readline()
    
    for lines in day0f:
        elemts = lines.split("\t")
        day0[elemts[1]]=elemts[8]


In [ ]:
## Get the other sas files:
some_path = os.path.split(os.path.realpath(sasfile_day0))[0]
p = subprocess.Popen(["ls", some_path] , stdout=subprocess.PIPE)
out, err = p.communicate()

tmp = str(out).split("'")
sasFiles = tmp[1].split("\\n") ## last one is empty

## Get the path for the output txt files:
txtFiles_path = os.path.split(os.path.realpath(some_path))[0]
tmpTxtFiles_path = "/".join([txtFiles_path, "tmp"])
os.makedirs(tmpTxtFiles_path, exist_ok=True)

newTmpTxtFiles = []
for sasFile in sasFiles:
    if re.search("sas7bdat", sasFile):
        ## create output file name + save in array to use later
        tmpf = sasFile.split(".")
        tmpfname = ".".join([tmpf[0], "tmp.txt"])

        newTmpTxt = "/".join([tmpTxtFiles_path,tmpfname])
        newTmpTxtFiles.append(newTmpTxt)

        ssfile = "/".join([some_path, sasFile])
        ## read in sas file
        with SAS7BDAT(ssfile) as sasf:
            df = sasf.to_data_frame()
        df.to_csv(newTmpTxt,sep="\t")


for newtextfiles in newTmpTxtFiles:
    ## read in
    with open(newtextfiles, "r") as tmpfile:
        ## create final output name:
        exSAS = os.path.split(os.path.realpath(newtextfiles))[1]
        tmpExsas = exSAS.split(".")
        fileNname = ".".join([tmpExsas[0], "txt"])
        convertedSAS = "/".join([txtFiles_path, fileNname])
        ## check each column name for something like date
        dateColumn = []
        first_line = tmpfile.readline()
        first_line = first_line.strip()
        headers = first_line.split("\t") 
        for i in range(0, len(headers)):
            if re.search("date", headers[i].lower()):
                dateColumn.append(i+1)

        ## write in new file the corrected dates:
        with open(convertedSAS, "w") as nwsas:
            nwsas.write(first_line)
            nwsas.write("\n")
            print(convertedSAS)
            
            ## correct date format
            for lines in tmpfile:
                lines = lines.strip()
                col = lines.split("\t")
                nwsas.write(col[1])
                for j in range(2, len(col)):
                    if j in dateColumn:
                        if re.match("\d\d\d\d-\d\d-\d\d", col[j]):
                            yr0, mh0, dy0 = day0[col[1]].split("-") ## hoping that the delimiter is - for all dates.
                            yr, mh, dy = col[j].split("-")
                         
                            newdate = date(int(yr), int(mh), int(dy)) - date(int(yr0), int(mh0), int(dy0))
                            nwsas.write("".join(["\t", str(newdate.days)]))
                        else:
                            nwsas.write("".join(["\t",col[j]]))
                    else:
                        nwsas.write("".join(["\t",col[j]]))
                nwsas.write("\n")
                    
                
                